In [4]:
import numpy as np
from tqdm import tqdm
import cv2
import os 
import imutils

In [6]:
def crop_image_based_on_extreme_points(input_img):
    # Convert the input image to grayscale and apply Gaussian blur for noise reduction
    gray_img = cv2.cvtColor(input_img, cv2.COLOR_RGB2GRAY)
    blurred_gray_img = cv2.GaussianBlur(gray_img, (3, 3), 0)

    # Threshold the blurred image to create a binary image, then perform erosion and dilation to remove noise
    binary_img = cv2.threshold(blurred_gray_img, 45, 255, cv2.THRESH_BINARY)[1]
    binary_img = cv2.erode(binary_img, None, iterations=2)
    binary_img = cv2.dilate(binary_img, None, iterations=2)

    # Find contours in the binary image and obtain the largest contour
    contours = cv2.findContours(binary_img.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)
    largest_contour = max(contours, key=cv2.contourArea)

    # Find the extreme points of the largest contour
    leftmost_point = tuple(largest_contour[largest_contour[:, :, 0].argmin()][0])
    rightmost_point = tuple(largest_contour[largest_contour[:, :, 0].argmax()][0])
    topmost_point = tuple(largest_contour[largest_contour[:, :, 1].argmin()][0])
    bottommost_point = tuple(largest_contour[largest_contour[:, :, 1].argmax()][0])

    # Add extra pixels for cropping margin and create a new cropped image
    ADD_PIXELS = 5
    cropped_img = input_img[topmost_point[1]-ADD_PIXELS:bottommost_point[1]+ADD_PIXELS,
                            leftmost_point[0]-ADD_PIXELS:rightmost_point[0]+ADD_PIXELS].copy()

    return cropped_img
